In [ ]:
import cloudscraper 
from bs4 import BeautifulSoup
import json
import pandas as pd
import sqlite3
import os 
import datetime as dt
import requests
import re

## Renda Fixa

In [ ]:
# Coletar URl para Renda Fixa
'''url = 'https://investidor10.com.br/renda-fixa/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')'''
# O site tem cloudflare, portanto vamos tentar com o Agent
'''
url = 'https://investidor10.com.br/renda-fixa/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')'''

: 

### Utilizar a API do site

In [ ]:
class Etl:
    '''
    Classe para realizar operações de ETL (Extract, Transform, Load) em dados financeiros.
    
    Métodos:
    - fillna_columns: Preenche valores NaN em colunas específicas com zero.
    - transform: Aplica transformações nos dados, incluindo o preenchimento de NaNs.
    - load: Carrega os dados transformados em um banco de dados SQLite.
    - extract: Extrai dados de uma fonte (a ser implementado).
    - __init__: Inicializa a classe com um scraper, uma lista para armazenar dados e um DataFrame vazio.
    
    Atributos:
    - scraper: Objeto para realizar requisições web.
    - all_data: Lista para armazenar todos os dados coletados.
    - page: Número da página atual para paginação.
    - df: DataFrame do pandas para manipulação de dados.
    
    '''
    def __init__(self):
        self.scraper = cloudscraper.create_scraper()
        self.all_data = []
        self.page = 1
        self.df = pd.DataFrame()
    
    
    def fillna_columns(self, df: pd.DataFrame, columns: list):
        """
        Preenche valores NaN em colunas específicas com zero.
        Se a coluna não existir, lança uma exceção personalizada.
        """
        for column in columns:
            try:
                if column in df.columns:
                    df[column].fillna(0, inplace=True)
                else:
                    raise KeyError(f"A coluna '{column}' não existe no DataFrame.")
            except Exception as e:
                print(f"Erro ao processar a coluna '{column}': {e}")
        
        return df


    def extract(self):
        
        while True:
            api_url = f"https://investidor10.com.br/api/fixed-incomes?page={self.page}&filter=&order="
        
            try:
                response = self.scraper.get(api_url)
                response.raise_for_status()
                
                data = response.json()
                
                # A API retorna um dicionário com a chave 'data', que contém a lista de itens.
                # Se 'data' estiver vazia, significa que chegamos ao fim da paginação.
                page_data = data.get('data', [])
                if not page_data:
                    print(f"Página {self.page} não tem mais dados. Paginação concluída.")
                    break # Sai do loop
                    
                self.all_data.extend(page_data) # Adiciona os dados da página atual à lista principal
                print(f"Página {self.page} coletada. {len(page_data)} itens encontrados.")
                
                self.page += 1 # Prepara para a próxima página
                
            except Exception as e:
                print(f"Ocorreu um erro ao coletar a página {self.page}: {e}")
                break # Sai do loop em caso de erro

        # Agora você tem todos os dados na variável `self.all_data`
        if self.all_data:
            print(f"\n Coletados {len(self.all_data)} títulos de renda fixa no total.")
            # Você pode inspecionar o primeiro item para ver a estrutura dos dados
            print("\n Exemplo do primeiro item coletado:")
            print(json.dumps(self.all_data[0], indent=4))
        else:
            print("Nenhum dado de renda fixa foi coletado.")
        
        
        # Colocar os Dados dentro do Dataframe
        self.df = pd.DataFrame(self.all_data)       
        
        return self.df
    
    def transform(self, data):
        columns_to_fillna = ['company_id','stock_bdr_id','slug','thumbnail'] # Colunas a preencher com zero
        self.fillna_columns(data, columns_to_fillna).info()
        
        return data

        
etl_ = Etl()
data = etl_.extract()
data = etl_.transform(data)


Página 1 coletada. 30 itens encontrados.
Página 2 coletada. 30 itens encontrados.
Página 3 coletada. 30 itens encontrados.
Página 4 coletada. 30 itens encontrados.
Página 5 coletada. 30 itens encontrados.
Página 6 coletada. 30 itens encontrados.
Página 7 coletada. 30 itens encontrados.
Página 8 coletada. 30 itens encontrados.
Página 9 coletada. 4 itens encontrados.
Página 10 não tem mais dados. Paginação concluída.

 Coletados 244 títulos de renda fixa no total.

 Exemplo do primeiro item coletado:
{
    "id": 5665,
    "name": "CRI Dasa IPCA + 10,20%",
    "slug": "cri-dasa-ipca-1020-5665",
    "type": "CRI",
    "fgc_guarantee": 0,
    "active": 1,
    "minimum_investment": 993.73,
    "financial_risk": "Baixo",
    "daily_liquidity": "No Vencimento",
    "taxation": "EXEMPT_INCOME",
    "indexer_profitability": "IPCA+",
    "type_profitability": "HYBRID",
    "value_profitability": "10.20",
    "redemption_period": "15/10/2032",
    "incomeable_type": "App\\Models\\Company",
    "ne

C:\Users\guilherme.dias\AppData\Local\Temp\ipykernel_6340\3288308198.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(0, inplace=True)


In [ ]:
data['stock_bdr_id'].unique()

array([ nan, 722.,  14.])

In [127]:


class Db:
    '''
    Classe para gerenciar a conexão com o banco de dados SQLite e operações básicas.
    '''
    def __init__(self):
        self.db_path = os.path.join(os.getcwd(), '..', '..', 'datawarehouse')
        self.db_file = os.path.join(self.db_path, 'datawarehouse.sqlite3')
        self.conn = sqlite3.connect(self.db_file) 
        self.cursor = self.conn.cursor()
        print(f"Conectado ao banco de dados SQLite")
    
    def drop_table(self, table_name: str):
        '''
        Remove uma tabela do banco de dados se ela existir.
        '''
        try:
            self.cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
            self.conn.commit()
            print(f"Tabela '{table_name}' removida com sucesso.")
        
        except Exception as e:
            print(f"Erro ao remover a tabela '{table_name}': {e}")
    
    def to_sql(self, df: pd.DataFrame, table_name: str, if_exists_: str):
        '''
        Carrega um DataFrame em uma tabela do banco de dados. Se a tabela já existir, ela será substituída.
        '''
        try:
            df['landing_date'] = dt.date.today()  # Adiciona a coluna 'landing_date' com a data e hora atual
            df.to_sql(table_name, self.conn, if_exists=if_exists_, index=False)
            print(f"Dados carregados na tabela '{table_name}' com sucesso.")
        
        except Exception as e:
            print(f"Erro ao carregar dados na tabela '{table_name}': {e}")
            
        finally:
            if self.conn:
                self.conn.close()
                print("Conexão com o banco de dados fechada.")
    
database = Db()
database.drop_table('acoes_indicadores')
#database.to_sql(data, 'renda_fixa', if_exists_='append')

Conectado ao banco de dados SQLite
Tabela 'acoes_indicadores' removida com sucesso.


# Ações

In [ ]:
class EtlAcoes:
    """
    ETL para coletar dados de ações do site Investidor10.
    """

    def __init__(self):
        self.base_url = "https://investidor10.com.br/acoes/rankings/maiores-valor-de-mercado/"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) "
                          "Chrome/120.0.0.0 Safari/537.36"
        }
        self.page = 1
        self.all_data = []
        self.df = pd.DataFrame()

    def extract(self):
        """
        Extrai todas as páginas até não encontrar mais linhas de ações.
        """
        while True:
            url = f"{self.base_url}?page={self.page}"
            print(f"Coletando página {self.page}...")

            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                print(f"Erro ao acessar página {self.page}: {response.status_code}")
                break

            soup = BeautifulSoup(response.content, "html.parser")
            rows = soup.find_all("tr")

            page_data = []
            for row in rows:
                info = {}
                ticker_span = row.find("span", class_="font-semibold text-[#14171F] group-hover:text-[#485063]")
                if not ticker_span:
                    continue
                info["ticker"] = ticker_span.get_text(strip=True)

                for td in row.find_all("td", class_="sorting"):
                    nome = td.get("data-name")
                    if nome:
                        valor = td.get_text(strip=True)
                        info[nome] = valor

                page_data.append(info)

            if not page_data:
                print(f"Página {self.page} não contém mais dados. Paginação concluída.")
                break

            self.all_data.extend(page_data)
            self.page += 1  # próxima página

        self.df = pd.DataFrame(self.all_data)
        print(f"Total de registros coletados: {len(self.df)}")
        return self.df

In [ ]:
acoes_inst = EtlAcoes()
acoes = acoes_inst.extract()

Coletando página 1...
Coletando página 2...
Coletando página 3...
Coletando página 4...
Coletando página 5...
Coletando página 6...
Página 6 não contém mais dados. Paginação concluída.
Total de registros coletados: 239


In [ ]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years,variation_30_days,variation_12_months,current_price,bazin_price,...,balance_net_revenue,growth_net_revenue_last_5_years,growth_net_profit_last_5_years,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months,dividend_yield_last_5_years,name_sector,name_subsector,name_segment
0,PETR4,"412,08 B",90,"5,13","0,99","416,06%","0,69%","-8,97%","R$ 30,80","R$ 131,90",...,"493,12 B","12,63%","65,59%","51,85 B","0,93","16,82%","25,73%","Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,"377,85 B",100,"9,34","1,92","178,90%","-3,26%","24,14%","R$ 37,11","R$ 27,32",...,"369,42 B","16,28%","23,81%","32,18 B",-,"7,41%","4,82%",Financeiro,Intermediários Financeiros,Bancos
2,VALE3,"267,57 B",70,"9,24","1,25","54,72%","4,86%","-2,07%","R$ 58,95","R$ 125,67",...,"209,60 B","0,10%","2,28%","31,09 B","0,45","7,61%","9,55%",Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,"229,34 B",100,"13,11","2,76","178,34%","-0,95%","43,47%","R$ 45,83","R$ 11,96",...,"50,13 B","30,59%","32,53%","1,38 B",-,"2,21%","2,23%",Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,"185,04 B",70,"12,54","2,00","7,23%","-4,16%","-4,88%","R$ 11,74","R$ 12,13",...,"91,72 B","9,46%","5,30%","17,52 B","0,03","8,87%","4,66%",Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,MTRE3,"389,25 M",70,"6,78","0,39","-62,66%","-4,66%","8,63%","R$ 3,68","R$ 8,05",...,"1,14 B","22,63%","10,35%","182,61 M","0,67","13,58%","9,60%",Consumo Cíclico,Construção Civil,Incorporações
235,CRPG5,"387,03 M",40,"-51,57","0,58","-21,66%","-18,30%","-53,63%","R$ 11,83","R$ 38,94",...,"732,15 M","0,02%",-,"103,90 M","0,01",-,"5,79%",Materiais Básicos,Químicos,Químicos Diversos
236,MEAL3,"363,69 M",40,"-2,76","0,39","-61,16%","-7,30%","-8,63%","R$ 1,27",R$ -,...,"2,11 B","12,88%",-,"297,72 M","0,58",-,-,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares
237,BHIA3,"309,03 M",20,"-0,17","0,20","-99,29%","-28,57%","-28,26%","R$ 3,25",R$ -,...,"28,24 B","-0,46%",-,"1,88 B","6,78",-,-,Consumo Cíclico,Comércio,Eletrodomésticos


In [ ]:
acoes.columns

Index(['ticker', 'enterprise_value', 'rate', 'p_l', 'p_vp',
       'variation_5_years', 'variation_30_days', 'variation_12_months',
       'current_price', 'bazin_price', 'bazin_upside', 'graham_price',
       'graham_upside', 'roe', 'net_margin', 'balance_net_profit',
       'balance_net_revenue', 'growth_net_revenue_last_5_years',
       'growth_net_profit_last_5_years', 'balance_availability',
       'gross_debt_net_worth', 'dividend_yield_last_12_months',
       'dividend_yield_last_5_years', 'name_sector', 'name_subsector',
       'name_segment'],
      dtype='object')

In [ ]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years,variation_30_days,variation_12_months,current_price,bazin_price,...,balance_net_revenue,growth_net_revenue_last_5_years,growth_net_profit_last_5_years,balance_availability,gross_debt_net_worth,dividend_yield_last_12_months,dividend_yield_last_5_years,name_sector,name_subsector,name_segment
0,PETR4,"412,08 B",90,"5,13","0,99","416,06%","0,69%","-8,97%","R$ 30,80","R$ 131,90",...,"493,12 B","12,63%","65,59%","51,85 B","0,93","16,82%","25,73%","Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,ITUB4,"377,85 B",100,"9,34","1,92","178,90%","-3,26%","24,14%","R$ 37,11","R$ 27,32",...,"369,42 B","16,28%","23,81%","32,18 B",-,"7,41%","4,82%",Financeiro,Intermediários Financeiros,Bancos
2,VALE3,"267,57 B",70,"9,24","1,25","54,72%","4,86%","-2,07%","R$ 58,95","R$ 125,67",...,"209,60 B","0,10%","2,28%","31,09 B","0,45","7,61%","9,55%",Materiais Básicos,Mineração,Minerais Metálicos
3,BPAC11,"229,34 B",100,"13,11","2,76","178,34%","-0,95%","43,47%","R$ 45,83","R$ 11,96",...,"50,13 B","30,59%","32,53%","1,38 B",-,"2,21%","2,23%",Financeiro,Intermediários Financeiros,Bancos
4,ABEV3,"185,04 B",70,"12,54","2,00","7,23%","-4,16%","-4,88%","R$ 11,74","R$ 12,13",...,"91,72 B","9,46%","5,30%","17,52 B","0,03","8,87%","4,66%",Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,MTRE3,"389,25 M",70,"6,78","0,39","-62,66%","-4,66%","8,63%","R$ 3,68","R$ 8,05",...,"1,14 B","22,63%","10,35%","182,61 M","0,67","13,58%","9,60%",Consumo Cíclico,Construção Civil,Incorporações
235,CRPG5,"387,03 M",40,"-51,57","0,58","-21,66%","-18,30%","-53,63%","R$ 11,83","R$ 38,94",...,"732,15 M","0,02%",-,"103,90 M","0,01",-,"5,79%",Materiais Básicos,Químicos,Químicos Diversos
236,MEAL3,"363,69 M",40,"-2,76","0,39","-61,16%","-7,30%","-8,63%","R$ 1,27",R$ -,...,"2,11 B","12,88%",-,"297,72 M","0,58",-,-,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares
237,BHIA3,"309,03 M",20,"-0,17","0,20","-99,29%","-28,57%","-28,26%","R$ 3,25",R$ -,...,"28,24 B","-0,46%",-,"1,88 B","6,78",-,-,Consumo Cíclico,Comércio,Eletrodomésticos


In [ ]:
def split_value_and_scale(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Separa uma coluna com valores tipo '427,98 B' em duas:
      - <coluna>_num   -> valor numérico (float)
      - <coluna>_scale -> escala (M, B ou None)
    """
    df = df.copy()  # evita mexer no DF original

    # Garante que a coluna é string
    df[column] = df[column].astype(str)

    # Extrai números
    df[f"{column}_num"] = (
        df[column]
        .str.extract(r"([\d\.,]+)")[0]  # pega só o grupo
        .replace(r"\.(?=\d{3}(?:\.|,))", "", regex=True)  # remove milhar
        .str.replace(",", ".", regex=True)  # vírgula decimal -> ponto
        .astype(float)
    )

    # Extrai escalas (M ou B)
    df[f"{column}_scale"] = df[column].str.extract(r"([MB])")

    return df

def convert_br_number(series: pd.Series) -> pd.Series:
    """
    Converte uma Series com números no formato brasileiro (ex: '1.423,42')
    para float no formato Python (ex: 1423.42).
    """
    return (
        series.astype(str)
        .str.replace('%','')
        .str.replace('R$ ','')
        .str.replace('.', '')   # remove separador de milhar
        .str.replace(',', '.') # troca vírgula decimal por ponto
        .str.replace('-','0') # troca o '-' por '0' para a conversão para float
        .astype(float)
    )



In [ ]:
money_columns = ["enterprise_value", "balance_net_profit", "balance_net_revenue"]

In [ ]:
# Colunas para limpeza
# Colunas em porcentagem
columns_with_percentage = [
    'variation_5_years','variation_30_days',
    'variation_12_months','bazin_upside', 
    'graham_upside', 'roe', 'net_margin',
    'growth_net_revenue_last_5_years',
    'growth_net_profit_last_5_years',
    'dividend_yield_last_12_months',
    'dividend_yield_last_5_years'
]

columns_with_real_sign = [
    'graham_price',
    'bazin_price',
    'current_price',
]

columns_with_B_sign = [
    'enterprise_value',
    'balance_net_profit',
    'balance_net_revenue',
    'balance_availability'    
]

columns_numeric_with_comma = [
    'p_l','p_vp','current_price',
    'bazin_price','graham_price',
    'gross_debt_net_worth',
    'current_price'
]

In [ ]:
for money_col in money_columns:
    acoes = split_value_and_scale(acoes, money_col)

In [ ]:
for comma in columns_numeric_with_comma:
    acoes[comma] = convert_br_number(acoes[comma])

ValueError: could not convert string to float: ''

In [ ]:
for perce in columns_with_percentage:
    acoes[perce] = convert_br_number(acoes[perce])
    acoes.rename(
        columns={perce:f'{perce}_percentage'},inplace=True
    )

In [ ]:
acoes

,ticker,enterprise_value,rate,p_l,p_vp,variation_5_years_percentage,variation_30_days_percentage,variation_12_months_percentage,current_price,bazin_price,...,dividend_yield_last_5_years_percentage,name_sector,name_subsector,name_segment,enterprise_value_num,enterprise_value_scale,balance_net_profit_num,balance_net_profit_scale,balance_net_revenue_num,balance_net_revenue_scale
0,PETR4,"414,96 B",90,5.16,1.00,413.19,1.37,7.10,310.0,131.90,...,25.73,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",414.96,B,77.82,B,493.12,B
1,ITUB4,"388,46 B",100,9.62,1.98,185.53,0.42,28.66,3821.0,27.32,...,4.82,Financeiro,Intermediários Financeiros,Bancos,388.46,B,43.82,B,369.42,B
2,VALE3,"265,94 B",70,9.18,1.24,55.27,4.16,1.81,5859.0,125.67,...,9.55,Materiais Básicos,Mineração,Minerais Metálicos,265.94,B,27.87,B,209.60,B
3,BPAC11,"239,67 B",100,13.67,2.88,184.55,3.06,49.08,478.0,11.96,...,2.23,Financeiro,Intermediários Financeiros,Bancos,239.67,B,13.41,B,50.13,B
4,ABEV3,"187,09 B",70,12.68,2.02,14.51,3.42,4.62,1187.0,12.13,...,4.66,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes,187.09,B,15.19,B,91.72,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,RNEW4,"387,40 M",30,2.88,0.28,83.10,3.92,53.33,98.0,0.00,...,0.00,Utilidade Pública,Energia Elétrica,Energia Elétrica,387.40,M,126.76,M,394.64,M
235,MEAL3,"375,14 M",40,2.85,0.40,59.32,4.38,8.39,131.0,0.00,...,0.00,Consumo Cíclico,Hoteis e Restaurantes,Restaurante e Similares,375.14,M,131.67,M,2.11,B
236,BHIA3,"333,76 M",20,0.19,0.22,99.23,23.70,24.68,351.0,0.00,...,0.00,Consumo Cíclico,Comércio,Eletrodomésticos,333.76,M,1.78,B,28.24,B
237,AVLL3,"310,57 M",20,5.19,2.31,0.00,8.62,49.36,159.0,0.00,...,0.00,Consumo Cíclico,Construção Civil,Incorporações,310.57,M,37.60,M,685.95,M


In [ ]:
acoes = acoes[['ticker', 'enterprise_value_num','enterprise_value_scale','rate', 'p_l', 'p_vp',
       'variation_5_years_percentage', 'variation_30_days_percentage', 'variation_12_months_percentage',
       'current_price', 'bazin_price', 'bazin_upside_percentage', 'graham_price',
       'graham_upside_percentage', 'roe_percentage', 'net_margin_percentage', 'balance_net_profit_num','balance_net_profit_scale',
       'balance_net_revenue_num','balance_net_revenue_scale', 'growth_net_revenue_last_5_years_percentage',
       'growth_net_profit_last_5_years_percentage', 'balance_availability',
       'gross_debt_net_worth', 'dividend_yield_last_12_months_percentage',
       'dividend_yield_last_5_years_percentage', 'name_sector', 'name_subsector',
       'name_segment']]

NameError: name 'acoes' is not defined

In [ ]:
database.to_sql(acoes, table_name='acoes_indicadores',if_exists_='append')

NameError: name 'database' is not defined

### Indicadores Fundamentalistas

- Coletar a Rentabilidade dos ultimos 10 anos
- Outros indicadores fundamentalistas 
    - Dy
    - margem ebit
    - ev/ebitda
    - p/ebitda
    - Roa
    - Divida
    - Liquida/Patrimonio
    - Liquidez Corrente
    - Cagr
    - Patrimonio/Ativos
    - Patrimonio Liquido
    - Valor de Firma
    - Valor de Mercado
    - Ativo Circulante
    - Divida Bruta
    - Divida Liquida
    - Liquidez Média Diária


In [86]:
def formatar_colunas(df):
    """
    Limpa e padroniza os nomes das colunas do DataFrame.
    """
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace(' ', '_', regex=False)
        .str.replace('[^a-z0-9_]', '', regex=True)
        .str.replace('no_total_de_papeis', 'n_total_de_papeis', regex=False)
    )
    return df

def limpar_percentual(valor):
    """
    Converte valores como '94,09%' em 0.9409 (float)
    """
    if isinstance(valor, str) and "%" in valor:
        valor = valor.replace("%", "").replace(",", ".").strip()
        try:
            return float(valor) / 100
        except ValueError:
            return None
    return valor


def limpar_valores(valor):
    """
    Extrai apenas o número grande e inteiro de strings no formato 'R$ 267,94R$ 267.937.617.000'
    """
    if isinstance(valor, str):
        match = re.findall(r'[\d\.]+', valor)
        if match:
            valor_limpo = match[-1].replace('.', '')
            try:
                return float(valor_limpo)
            except ValueError:
                return None
    return valor


# --- Web scraping ---
ticker = 'vale3'
url_acoes = f'https://investidor10.com.br/acoes/{ticker}/'

headers = {'User-Agent': 'Mozilla/5.0'}  # evita bloqueio
soup = BeautifulSoup(requests.get(url_acoes, headers=headers).content, "html.parser")

dados = {}
a = soup.find_all('div', attrs={'class': 'table grid-3'})

for cel in a:
    for bloco in cel.find_all('div', class_='cell'):
        titulo = bloco.find('span', class_='title')
        valor = bloco.find('span', class_='value')
        if titulo and valor:
            dados[titulo.get_text(strip=True)] = valor.get_text(strip=True)

df = pd.DataFrame([dados])

# --- Formata nomes das colunas ---
df = formatar_colunas(df)

# --- Limpa os valores numéricos ---
columns_to_format = [
    'valor_de_mercado', 'valor_de_firma', 'patrimonio_liquido',
    'n_total_de_papeis', 'ativos', 'ativo_circulante',
    'divida_bruta', 'divida_liquida', 'disponibilidade','liquidez_media_diaria'
]

for col in columns_to_format:
    if col in df.columns:
        df[col] = df[col].apply(limpar_valores)

# --- Limpeza percentual ---
percent_cols = ['free_float', 'tag_along']
for col in percent_cols:
    if col in df.columns:
        df[col] = df[col].apply(limpar_percentual)
        df.rename(columns={col:f'{col}_percentage'},inplace=True)
        
print(df.T)

                                        0
valor_de_mercado           269934780000.0
valor_de_firma             336230780000.0
patrimonio_liquido         214175000000.0
n_total_de_papeis            4539007000.0
ativos                     493226000000.0
ativo_circulante            95964000000.0
divida_bruta                97383000000.0
divida_liquida              66296000000.0
disponibilidade             31087000000.0
segmento_de_listagem         Novo Mercado
free_float_percentage              0.9409
tag_along_percentage                  1.0
liquidez_media_diaria         926215000.0
setor                   Materiais Básicos
segmento               Minerais Metálicos


In [117]:
soup.find_all('p')

[<p class="title">
                     Se você tivesse investido <span id="value-simulator">R$ 1.000,00</span> <a class="change-value-simulator" data-type="quotation" href="javascript:void(0);">(alterar)</a> há <span class="simulation_period">1 ano</span> <a class="change-value-dt-simulator" data-type="quotation" href="javascript:void(0);">(alterar)</a>, hoje você teria: 
                         <span id="result-simulator">R$ 1.000,00</span><span class="showOnAdjusted">*</span>
 </p>,
 <p class="description-text" style="font-size: 15px">
                 A fórmula do Preço Justo foi criada por Benjamin Graham para identificar ações com potencial de valorização ou que estão sendo negociadas por menos do que valem. Graham foi um dos maiores investidores da história e mentor de Warren Buffet.
             </p>,
 <p class="title">Preço atual</p>,
 <p class="title">Preço Justo</p>,
 <p class="title">
                         Upside
                         <img alt="Potencial de rentabili

In [100]:
# ...existing code...
legend_data = []

for div in soup.find_all('div', attrs={'class': 'legend-container'}):
    for detail in div.find_all('div', attrs={'class': 'legend-details'}):
        name_tag = detail.find('p', attrs={'class': 'legend-name'})
        if name_tag:
            legend_data.append(name_tag.get_text(strip=True))

print(legend_data)
# ...existing code...

[]


In [126]:
# --- Parte 1: Preparação do ambiente ---
import asyncio
import nest_asyncio

# Permite reusar o loop já em execução no Jupyter
nest_asyncio.apply()

# Corrige bug do Windows com subprocessos do asyncio
if isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

print("✅ Loop configurado com sucesso! Pode seguir para a próxima célula.")

# --- Parte 2: Extração de dados renderizados ---
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def scrape():
    async with async_playwright() as p:
        # abre o navegador headless (sem interface)
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        ticker = 'vale3'
        url_acoes = f'https://investidor10.com.br/acoes/{ticker}/'
        await page.goto(url_acoes, wait_until="networkidle")

        # pega o HTML já renderizado (com JS executado)
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")

        # encontra os elementos de legenda
        items = soup.find_all("div", class_="legend-details")

        dados = []
        for item in items:
            nome = item.find("p", class_="legend-name").text.strip()
            receita = item.find("p", class_="legend-revenue").text.strip()
            dados.append({"Nome": nome, "Receita": receita})

        await browser.close()
        return dados
# executa a função e exibe os dados
dados = await scrape()
dados

✅ Loop configurado com sucesso! Pode seguir para a próxima célula.


NotImplementedError: 